# **The Molecular Treasure Hunt: Part 3: Understanding the meaning of your docking scores**

*An &#8491;ngstrom sized adventure by Sarah Harris (Leeds Physics) and Geoff Wells (UCL Pharmacy)*

"*Space is big. You just won't believe how vastly, hugely, mind-bogglingly big it is.”* Douglas Adams, The Hitchhiker's Guide to the Galaxy.

# Introduction
In Part 2 of your treasure hunt, you obtained poses for your ligand library against a selection of protein conformers. This is a huge amount of data, and to make any sense of it we need to make decisions about which are the "best" hits. This is currently a somewhat subjective (and flawed) process, and is the subject of international docking competitions in which research groups attempt to predict experimentally determined protein-ligand complexes, and their affinities (see https://www.capri-docking.org/).

# What Part 3 will reveal to you!
You will probably find that you have far too much information to analyse visually. In this part we process and manipulate our results, to take us closer to the heart of our quest!!

## A few things to think about before we start...

## Drug design considerations
In drug design programmes, "fragment hits" typically have high $\mu M$ to low $m M$ $K_{i}$ values, where the $K_{i}$ is the equilibrium dissociation constant for the ligand, and represents the probability that the ligand is bound or unbound at a given concentration:

\begin{equation*}
K_{i}   = \frac{[L][P]}{[LP]}
\end{equation*}

where [L] is the concentration of the ligand, [P] is the concentration of the protein and [LP] is the concentration of the ligand-protein complex.

Fragment hits then need to undergo further chemical refinement to produce a putative drug (e.g. via fragment growth or linking).

"Hit" compounds tend to be larger molecules that we consider promising and worthy of further structure-activity relationship studies. For these, we might expect $K_{i}$ values in the low $\mu M$ range. Further along the development pipeline, we have "lead" compounds, which may progress to preclinical development. We may expect $K_{i}$ values in the low $n M$ range for these molecules (depending on the target and its tractability).

How close do we get in our virtual study???

## Why thermodynamics matters
One of the major uncertainties is how we interpret our docking scores in terms of a dissociation constant between the protein and the ligand. This is due to the difficulty in calculating thermodynamic properties (which are ensemble averages over around Avogadro's number of molecular entities) from single snapshots obtained by a dock. Moreover, there are additional complications such as induced fit of the receptor, environmental conditions (e.g. salt concentration) and interactions with water. The problem is that what we need for affinity is **free energy**, whereas what we can calculate from a single static protein structure is just the **energy** because the **entropy** term is missing when we don't include all possible protein, ligand and solvent conformations. This is currently impossible to compute. 

If you find this difficult to understand, you are not alone, and this issue is one of the concepts we explore in our treasure hunt when we consider multiple protein receptor conformations. Vina_BE is in the units of KCal/mol (http://autodock.scripps.edu/faqs-help/faq/how-autodock-4-converts-binding-energy-kcal-mol-into-ki). 

To convert from Vina_BE to an estimate of the ligand dissociation constant ($K_{i}$), that we call Vina_Ki in this notebook, we use the following equation:


\begin{equation*}
K_{i}   = e^{\Delta G/RT}
\end{equation*}

Here we interpret $\Delta$G as being equivalent to the Vina_BE, R is the ideal gas constant and T is the temperature. We need the temperature to be in Kelvin (K) and R to be in Kcal/mol/K. Because the $K_{i}$ is a function of the free energy, $\Delta$G, it tells us about probabilities. For example, a dissociation constant ($K_{i}$) of $10^{-3} M$ means that for each unbound ligand, 1000 are bound to the protein. Conversely, a dissociation constant of $10^{3} M$ means that for each 1000 unbound ligand molecules only one is bound to the protein receptor. Note that below we quote Vina_Ki values in $\mu M$ because ligand $K_{i}$ values often span the $m M$ to $n M$ range of concentrations.

It would also be possible to consider the association constant ($K_{a}$) where:

\begin{equation*}
K_{a}   = \frac{1}{K_{i}}
\end{equation*}

Which of course, from thermodynamics, implies that:

\begin{equation*}
K_{a}   = e^{-\Delta G/RT}
\end{equation*}

**Generally, all docking scores should be viewed as a rank, rather than a prediction of binding free energy.**

**A note on experimental binding assays**: IC$_{50}$ values (the ligand concentration producing a 50% change in an assay) are more prevalent than $K_{i}$ values in the literature because they can be measured in high-throughput experiments, sometimes using liquid handling robotics. An important distinction between $K_{i}$ and IC$_{50}$ is that the latter are assay dependent and affected by, for example, the concentration of the protein and ligand. While equilibrium dissociation constants ($K_{d}$) can be determined using direct binding techniques such as isothermal titration calorimetry, microscale thermophoresis or surface plasmon resonance methods, these methods can be lower throughput and more time consuming.

In [ ]:
#This imports the python modules that we need for the notebook...
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
from glob import glob
import seaborn as sns
import scipy.cluster.hierarchy as shc

In [ ]:
#This function extracts the docking and properties info from the various results files in our directory
def extract_docking_results_from_mol2(filename):
    
    DockedFilePath = filename
    DockedFilePrefix = DockedFilePath.split(".")[0]
    DockedFile = DockedFilePrefix.split("VinaResults_")[1]
    Protein_conf = []
    Name = []
    Vina_BE = []
    MW = []
    clogP = []
    nonHatoms = []
    Vina_LE = []
    Vina_LipE = []
    Vina_Ki = []
    CC = []
    HydroC = []
    HBonds = []
    SaltB = []
    PiPi_Par = []
    PiPi_Perp = []
    Pi_Cat = []
    HalB = []

    with open(DockedFilePath, 'r') as f:
        for line in f:
            if line.startswith("########## Name:"):
                Name.append(str(line.split()[2]))
                Protein_conf.append(DockedFile)  
            if line.startswith("########## Vina_BE:"):
                if float(line.split()[2])<0:
                    Vina_BE.append(float(line.split()[2]))
                    #Here the gas constant R is in the units Kcal/mol/K and the temperature is 300 K 
                    #Ki (the inhibitor dissociation constant) is in micromolar units
                    Vina_Ki.append((math.exp((float(line.split()[2]))/(0.001987*300))*1000000))
                else:
                    #Here we set the Vina_BE and Vina_Ki to 0 if the values are greater than 0 (which may be an indication that the compound does not fit in the binding site and/or grid box)
                    #You should look out for these zero values in your graphs!
                    Vina_BE.append(0)
                    Vina_Ki.append(0)
                
                    
                    
            if line.startswith("########## MW:"):
                MW.append(float(line.split()[2]))
            if line.startswith("########## clogP:"):
                clogP.append(float(line.split()[2]))
            if line.startswith("########## nonHatoms:"):
                nonHatoms.append(int(line.split()[2]))
            if line.startswith("########## Vina_LE:"):
                Vina_LE.append(float(line.split()[2]))
            if line.startswith("########## Vina_LipE:"):
                Vina_LipE.append(float(line.split()[2]))
            if line.startswith("########## Close_contacts:"):
                CC.append(int(line.split()[2]))
            if line.startswith("########## Hydrophob_contacts:"):
                HydroC.append(int(line.split()[2]))
            if line.startswith("########## HBonds:"):
                HBonds.append(int(line.split()[2]))
            if line.startswith("########## Salt_Bridges:"):
                SaltB.append(int(line.split()[2]))
            if line.startswith("########## Pi-Pi_Parallel:"):
                PiPi_Par.append(int(line.split()[2]))
            if line.startswith("########## Pi-Pi_Perpendicular:"):
                PiPi_Perp.append(int(line.split()[2]))
            if line.startswith("########## Pi-Cation:"):
                Pi_Cat.append(int(line.split()[2]))
            if line.startswith("########## Halogen_Bonds:"):
                HalB.append(int(line.split()[2]))
            
    
    #The units of Vina_BE are kcal/mol. R = 1.987 cal/mol/K.
    df = pd.DataFrame(list(zip(Protein_conf, 
                               Name, 
                               Vina_BE, 
                               Vina_Ki, 
                               MW, 
                               nonHatoms, 
                               clogP, 
                               Vina_LE, 
                               Vina_LipE, 
                               CC, 
                               HydroC, 
                               HBonds, 
                               SaltB, 
                               PiPi_Par, 
                               PiPi_Perp, 
                               Pi_Cat, 
                               HalB)), 
                   columns =['Protein_Conf', 
                             'Ligand_Name', 
                             'Vina_BE', 
                             'Vina_Ki', 
                             'MW', 
                             'nonHatoms', 
                             'clogP', 
                             'Vina_LE', 
                             'Vina_LipE', 
                             'Close_Contacts', 
                             'Hphob_Contacts', 
                             'HBonds', 
                             'Salt_Bridges', 
                             'Pi-Pi_Par', 
                             'Pi-Pi_Perp', 
                             'Pi-Cation', 
                             'Hal_Bonds'])
            
    return(df)
    f.close()

# Creating 'pandas dataframes' from our docking data
Here we are going to read in the results from our results folder, and process them to produce a special python object called a **pandas dataframe**. This python data structure effectively represents your data in a table, which then helps us to extract the information we need and to make plots of our data.

In [ ]:
# This defines our receptor structures (all of these files happen to have the same prefix, the starting structure (xal structure) is xxx_00.pdb)
path_results_filenames = glob('MTH_Results/VinaResults_*.mol2')
path_results_filenames.sort()
dataframes_list = []
for path_results_filename in path_results_filenames:
    df_out = extract_docking_results_from_mol2(path_results_filename)
    dataframes_list.append(df_out)
total_dataframes = pd.concat(dataframes_list, ignore_index=True)    

Here we take a look at the Pandas dataframe - it looks neat and it is easy to understand, but it is large. The aim of this stage is to visualise and understand what you have discovered.

In [ ]:
#Save the data frame in a csv file format
total_dataframes.to_csv('rescored_docking_results.csv', index=False)

In [ ]:
#This lists all of our results...
total_dataframes

# Plotting our data and saving our graphs
Now we try to understand our docking results by plotting some summary graphs and charts. This is important, because it is impossible to know how we should best choose hits unless we first assess the trends across the whole data set. Notice that we consider data distributions in our treasure hunt more often than averages.

In the first example we save the graphical output as a picture file that you can use in presentations and reports. There is a line in the code: "plt.savefig('Vina_BE_vs_compounds.png')" that you can paste into other cells in the notebook if you want to save the chart or graph. Remember to change the file name to something relevant to your output!

## Looking at the whole dataset
First we will look at the Vina_BE for each ligand, across each of the protein conformers. Having around 2000 ligands means that this graph is very busy. We concluded this this representation of our data wasn't very informative.

In [ ]:
#We are plotting binding energy for each ligand: each point represents one ligand and one protein conformation.
plt.figure(figsize=(16,12), dpi= 80)
plt.plot(total_dataframes.index, total_dataframes.Vina_BE, 'o', color='black');
plt.title('Vina_BE (in KCal/mol) for each ligand', fontsize=22)
plt.xticks(fontsize=12, rotation=90)
plt.xlabel('Ligand Index', fontsize=12)
plt.ylabel('Vina_BE (KCal/mol)', fontsize=12)
plt.yticks(fontsize=12)
#The next line saves this graph as a png file in the current folder
plt.savefig('Vina_BE_vs_compounds.png')
plt.show()

## Looking at distributions
As we have so much data, it is probably best to take a look at distributions, rather than the actual values (as we tried above). Here we plot:
- Vina_BE (KCal/mol)
- Number of hydrogen bonds formed between the protein and the ligand
- Number of hydrophobic contacts between the protein and the ligand 

If you see a lot of Vina_BE values close to 0 KCal/mol it may tell you that your grid box is too small and the ligands are protruding from the box!

In [ ]:
#A series of histograms to tell us about the distributions of our docking results...
total_dataframes.hist('Vina_BE', bins=20)
total_dataframes.hist('HBonds', bins=10)
total_dataframes.hist('Hphob_Contacts', bins=10)

We can use our special pandas dataframe to sort the results by Vina_BE, putting the lowest (e.g. most favourable) energy first. We can the visualise this version of our dataframe that is "sorted by Vina_BE" to check we understand what we are plotting (this may not be included in the pdf version).

In [ ]:
sort_by_BE = total_dataframes.sort_values('Vina_BE')
sort_by_BE

## We always see this 'S'-shaped profile!
We can now produce a ranked (waterfall) plot of the Vina_BE vs compound (shown as index in the plot for clarity). We have observed that this has a very characteristic "S" shape for all ligand libraries and protein receptors we have tried so far. We think that the best "hits" can only be those in the first steep region of the curve, because in the shallow region a small change in binding energy (e.g. Vina_BE in KCal/mol) produces a large change in rank for the compound.

In [ ]:
sort_by_BE.reset_index(drop=True, inplace=True)
sort_by_BE.reset_index().plot(kind='scatter', x='index', y='Vina_BE', s=10)

Here we plot the Vina_BE as a function of molecular weight (and the points are coloured by the clogP value, that estimates the lipophilicity of the ligand).

You might expect to see that larger ligands have lower (better) Vina_BE scores because they can form more contacts with the protein.  *However*, these compounds may also be less *efficient* and have a lower binding energy per unit size (this is measured in the Vina_LE score, which is the Vina_BE divided by the number of non-hydrogen atoms).

These trends might be more obvious for compound libraries that span a large range of molecular weights e.g. the FDA approved drug library.

In [ ]:
plt.figure(figsize=(10,8), dpi= 80)
plt.title('Vina_BE (in KCal/mol) as a function of molecular weight MW (g/mol)', fontsize=14)
sns.scatterplot(data=sort_by_BE, x='MW', y='Vina_BE', hue='clogP', s=10, edgecolor="black", linewidth=0.1)

## Looking at correlations
This next plot shows correlations in the numerical variables in the docking dataset.

Can you see any interesting correlations, or are there variables that are not as correlated as you might have hoped? Can you explain the correlations that you see?

Sometimes you can see blank rows and columns if there are no data to correlate (e.g. if none of the compounds in the library form a particular type of interaction with the protein)

In [ ]:
independent_variables = total_dataframes.drop(['Protein_Conf', 'Ligand_Name', 'MW', 'Vina_LE', 'Vina_LipE', 'Vina_Ki', 'Pi-Pi_Par', 'Pi-Pi_Perp'], axis=1)
plt.figure(figsize=(12,10), dpi= 80)
sns.heatmap(independent_variables.corr(), xticklabels=independent_variables.corr().columns, yticklabels=independent_variables.corr().columns, cmap='RdYlGn', center=0, annot=True)
plt.title('Correlogram of Docking Variables', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

## Observing the importance of dynamics and protein conformation
The next very informative plot is a version of our 'waterfall' plot for our ranked compounds, but now seperated so we can see how it depends on the protein receptor conformation.

We will see whether some protein conformations are better or worse at binding ligands. This plot is giving us direct information about the importance of dynamics (or other structural variations of the protein) on our docking results.

In [ ]:
plt.figure(figsize=(10,8), dpi= 80)
sns.scatterplot(data=sort_by_BE, x='Ligand_Name', y='Vina_BE', hue=sort_by_BE.Protein_Conf.tolist(), s=15, linewidth=0.1)

## Correlations with physicochemical properties
As well as binding "energies" (Vina_BE), other properties of ligands are known to be important in molecular recognition. Pharmaceutical intution suggests that for a compound to engage in a molecular recognition event it must form some **hydrogen bonds** (which are distance and angle dependent interactions) or **electrostatic** interactions with the target, otherwise they will not be sufficiently chemically specific to the binding site. 

We can determine whether H-bonds are important to the Vina_BE, and consider also the other important physicochemical property **clogP**, which is an estimate of the octanol/water partition coefficient (a measure of the lipophilicity of the compound). This is important because how tells you how likely a drug is to cross a cell membrane, which relates to its bioavailability. One of *Lipinski's Rules of 5* is that logP should be no greater than 5 for an orally bioavailable drug. 

We can sometimes see that relatively non-polar compounds (clogP > 3-4) form few H-bonds in this type of plot, as expected. These trends can be more apparent with compound libraries that span a wide range of clogP values (e.g. the FDA approved drug library).

In [ ]:
plt.figure(figsize=(8,6), dpi= 80)
sns.scatterplot(data=sort_by_BE, x='HBonds', y='Vina_BE', hue='clogP')

This next series of plots allow us to correlate the various independent properties of our compounds with each other, and to see how this is affected by protein receptor conformation. *Note: we have to remove H-bonds from this analysis because the large number of compounds with no H-bonds affects the distributions.*

The plots on the diagonals are histograms showing how the property is distributed for each protein conformation. So physicochemical properties should be the same (clogP, non-hydrogen atoms), but the Vina_BE distribution will change for each protein conformer.

In [ ]:
#First drop the columns that contain too many zeros for this type of plot...
no_HBonds = total_dataframes.drop(['MW', 
                                   'Vina_LE',
                                   'Vina_LipE',
                                   'Vina_Ki', 
                                   'Close_Contacts', 
                                   'Hphob_Contacts', 
                                   'HBonds', 
                                   'Salt_Bridges', 
                                   'Pi-Pi_Par', 
                                   'Pi-Pi_Perp', 
                                   'Pi-Cation', 
                                   'Hal_Bonds'], axis=1)
plt.figure(figsize=(16,10), dpi= 80)
sns.pairplot(no_HBonds, kind="scatter", hue="Protein_Conf", plot_kws=dict(s=10, edgecolor="black", linewidth=0.1))
plt.show()

## Distributions and protein conformational dynamics
We can also use "violin plots" to understand the distributions of scored energies for each different protein conformation. Note that the most prestigious scientific journals now insist on graphs that show distributions rather than bar charts, which do not. Why do you think this is so important?

Are there any protein conformers that look like outliers, or are they all the same?

In [ ]:
# Draw Violin Plot
plt.figure(figsize=(13,10), dpi= 80)
sns.violinplot(x='Protein_Conf', y='Vina_BE', data=total_dataframes, density_norm='width', inner='quartile')

# Decoration
plt.title('Violin Plot of Vina_BE by Protein Conformation', fontsize=22)
plt.xticks(fontsize=10, rotation=90)
plt.yticks(fontsize=10)
plt.show()

# Congratulations!!
You have completed the third part of your treasure hunt, in the next part we will save your greatest 'hits' and make files that help us to look at the docked conformations and how the ligands are predicted to bind to the protein.

"*Forty-two, said Deep Thought, with infinite majesty and calm.*" Douglas Adams, The Hitchhiker's Guide to the Galaxy

Good luck!!

Sarah and Geoff

(an "out-of-offices studios" $O^{3}S$ production)